Import libraries

In [ ]:
import pandas as pd
import numpy as np
import pickle
import datetime

This function returns current date and time in this format "MMDDYYHHMMSS".

In [ ]:
def getCurrentDateTime():
    # Get the current date and time
    current_datetime = datetime.datetime.now()

    # Extract the individual components
    month = current_datetime.strftime("%m")  # MM
    day = current_datetime.strftime("%d")  # DD
    year = current_datetime.strftime("%y")  # YY
    hour = current_datetime.strftime("%H")  # HH
    minute = current_datetime.strftime("%M")  # MM
    second = current_datetime.strftime("%S")  # SS

    # Concatenate the components to form the desired format
    formatted_date_time = f"{month}{day}{year}{hour}{minute}{second}"

    return formatted_date_time

A function that read input data as file name to load in the datasource folder and returns a datatfame of the file.
In this case the file type is in xlsx.

In [ ]:
def readFile():
    file_found = False
    while not file_found:

        try:
            file_name = input("Enter file name to load: ")
            file = f"datasource/{file_name}.xlsx"
            file_df = pd.read_excel(file)
            print(f"Successfully loaded file: {file_name}")
            return file_df
        except:
            print(f"{file_name} not found.")

Read raw file excel data and assign the data frame data to raw variable.

In [ ]:
raw = readFile()

Load pickle file data and store it to dnc_copy variable

In [ ]:
# Open pickled file
with open('session.pkl', 'rb') as f:
    data = pickle.load(f)

dnc_copy = data['DNCU']

Fill NaN's with 0

In [ ]:
dnc_copy["value"] = dnc_copy["value"].fillna(0)
dnc_copy["value"] = dnc_copy["value"].astype(np.int64)

Show infos for raw data source and dnc

In [ ]:
raw.info()
dnc_copy.info()

Filters out data in UNIQUE column that is in dnc.
Exports the result in excel file inside the output folder.

In [ ]:
in_dnc = raw[raw['Work Phone'].isin(dnc_copy['value'])]
dnc_file_name = f"DNC_{getCurrentDateTime()}"
dnc_file = f"output/DNC/{dnc_file_name}.xlsx"
display(in_dnc)
in_dnc.to_excel(dnc_file, index=False)

Filters out data in UNIQUE column that is not in dnc.
Exports the result in excel file inside the output folder.

In [ ]:
not_in_dnc = raw[raw['Work Phone'].isin(dnc_copy['value']) == False]
not_dnc_file_name = f"NOT_DNC_{getCurrentDateTime()}"
not_dnc_file = f"output/NOT_DNC/{not_dnc_file_name}.xlsx"
display(not_in_dnc)
not_in_dnc.to_excel(not_dnc_file, index=False)

Get the summary of the data and display as a dataframe.

In [ ]:
total_leads = raw.shape[0]
unique = raw['Work Phone'].nunique()
duplicates = total_leads - unique
dnc_count = in_dnc.shape[0]
good_leads = not_in_dnc.shape[0]

total_percent = "100%"
unique_percent = f"{round((unique/total_leads)*100)}%"
dup_percent = f"{round((duplicates/total_leads)*100)}%"
dnc_percent = f"{round((dnc_count/total_leads)*100)}%"
gl_percent = f"{round((good_leads/total_leads)*100)}%"

# initialize list of lists 
summary_data = [['TOTAL LEADS', total_leads, total_percent], ['UNIQUE', unique, unique_percent], ['DUPLICATES', duplicates, dup_percent], ['DNC', dnc_count, dnc_percent], ['GOOD LEADS', good_leads, gl_percent]] 
# Create the pandas DataFrame 
summary_df = pd.DataFrame(summary_data, columns=['Description', 'Count', 'Percentage']) 
summary_df.style.hide(axis="index")
